In [ ]:
%matplotlib inline

import torch    as t
import torch.nn as nn

from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
from itertools import islice

# TP Reseaux de neurones et Convolution

Dans ce TP, vous utilisez le formalisme le plus haut niveau de pyTorch (très proche de Keras), qui masque tous les détails

# Partie 0 - fonction de visualisation (a ne pas lire)

In [ ]:
def visualize_pytorch_classifier(X, y, predict=None,**kwargs):
    X_ = X.data.numpy()
    y_ = y.data.numpy()
    
    ax = plt.gca()
    
    # Plot the training points
    ax.scatter(X_[:, 0], X_[:, 1], c=y_, s=30, cmap='rainbow',
               clim=(y_.min(), y_.max()), zorder=3)
    ax.axis('tight')
    #ax.axis('off')
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    if predict:
        xx, yy = np.meshgrid(np.linspace(*xlim, num=200),
                             np.linspace(*ylim, num=200))
        xxyy   = np.c_[xx.ravel(), yy.ravel()]
        Z      = np.array([predict(Variable(t.from_numpy(d)).float(),**kwargs).data.numpy()
                           for d in xxyy]).reshape(xx.shape)

        # Create a color plot with the results
        n_classes = len(np.unique(y.data.numpy()))
        contours = ax.contourf(xx, yy, Z, alpha=0.3,
                               levels=np.arange(n_classes + 1) - 0.5,
                               cmap='rainbow', #clim=(y_.min(), y_.max()),
                               zorder=1)

        ax.set(xlim=xlim, ylim=ylim)


# Partie 1 - Régression Logistique

In [ ]:
# On reprend les memes donnees que dans le TP précédent
# On va apprendre une régression logistique dessus
n = 400

# On crée des donnees non separables lineairement:
X = Variable(t.randn(n,2))
y = X[:,0]**2-2*X[:,1]>0
y = y.float()

visualize_pytorch_classifier(X,y)

In [ ]:
net = nn.Sequential(
    nn.Linear(2, 1),   # Couche lineaire z = w0+w1.x1+x2.x2.   Les parametres wj sont initialises aleatoirement
    nn.Sigmoid()       # fonction de transfert sigmoide
)

fonction_de_cout = t.nn.BCELoss()  # fonction de cout de la regression logistique: y.log(p)+(1-y).log(1-p)

In [ ]:
# prediction sur l'exemple 0:
y_pred = net(X[0])
print( y_pred )

# cout
print( fonction_de_cout( y_pred,y[0]) )

In [ ]:
# Tous les parametres sont stockes dans net.parameters
print(list( net.parameters() ))

In [ ]:
def simple_learn(X,y,net,fonction_de_cout,ntrials=500,learning_rate = 0.05):

    for trial in range(ntrials):
        i = np.random.randint(0,n)
        xi= X[i]

        y_pred = net(xi)
        cout   = fonction_de_cout(y_pred, y[i])

        cout.backward()

        for param in net.parameters():
            param.data -= learning_rate * param.grad.data

        net.zero_grad()

In [ ]:
simple_learn(X,y,net,fonction_de_cout,ntrials=500,learning_rate = 0.05)
visualize_pytorch_classifier(X,y,net)

# Partie 2 - Réseau de neurones multi-couches

Ici, on va recoder avec `nn.Sequential` le réseau de neurones du TP précédent... mais en beaucoup moins de lignes de code !

In [ ]:
# On crée le meme réseau (a 2 couches cachées) que durant le dernier TP
net2 = nn.Sequential(
    nn.Linear(2, 3),
    nn.Sigmoid(),
    nn.Linear(3,1),
    nn.Sigmoid()
)

# et on applique la meme procédure d'apprentissage
simple_learn(X,y,net2,fonction_de_cout,ntrials=1000,learning_rate = 0.5)

# et on visualise
visualize_pytorch_classifier(X,y,net2)

#### Q: Affichez les poids des différents neurones
#### Q: Construisez un réseau à 3 couches cachées et deux neurones par couche cachée, et entrainez-le de nouveau

#### Créez une base de test `X_test` et `y_test` en utilisant la même procédure de création de données que pour `X` et `y`, et mesurez l'erreur de classification comise par ces réseaux sur ce jeu de test